# CUDA编程模型---基于ARM平台的Jetson NANO存储单元调用

Jetson系列（包括TX1，TX2，Xavier，NANO等）用的都是SoC芯片，CPU和GPU集成在一个芯片上，自然用的是同一个内存，因此GPU可以直接访问内存上的数据（100多GB/s）而不用受到PCIE的限制（10多GB/s)。

因此，在CUDA编程中可以舍弃cudaMemcpy系列函数（相当于在同一个内存上徒劳地复制了一遍），转而使用zero copy或者统一内存unified memory
今天的课程将介绍，在NANO上使用Pinned Memory加速程序

CUDA应用程序可以使用各种类型的内存缓冲区，例如设备内存，可分页的主机内存，固定内存和统一内存. 即使将这些内存缓冲区类型分配在同一物理设备上，每种类型也具有不同的访问和缓存行为，如下图所示. 选择最合适的内存缓冲区类型对于有效执行应用程序很重要.
![memory_type](memory_type.png)

先使用老的编译下 ，[matrix_mul_old.cu](matrix_mul_old.cu)文件， 
如果遇到麻烦，请参考[result_old.cu](result_old.cu)

In [17]:
!make -f Makefile_mlu_old

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80 matrix_mul_old.cu -o ./matrix_mul_old


In [18]:
!./matrix_mul_old

GPU Time = 1.34502 ms.
CPU Time = 3846.52 ms.
Pass!!!


接下来，我们就修改[matrix_mul.cu](matrix_mul.cu)文件，去掉```cudaMalloc()``` 和 ```cudaMemcpy()```，而采用统一内存的方法。  
如果遇到麻烦，请参考[result1.cu](result1.cu)

编译，并执行程序

In [19]:
!make

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80 matrix_mul.cu -o ./matrix_mul


In [20]:
!./matrix_mul

GPU Time = 2.75894 ms.
CPU Time = 3889.41 ms.
Pass!!!


利用nvprof查看性能

In [13]:
#!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

!/usr/local/cuda/bin/nsys nvprof --print-api-trace  ./matrix_mul

The --print-api-trace  switch is ignored by nsys.

GPU Time = 3.69034 ms.
CPU Time = 3960.84 ms.
Pass!!!
Generating '/tmp/nsys-report-bc6c.qdstrm'
[1/7] [========================100%] report1.nsys-rep
[2/7] [========================100%] report1.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/04_4.2/report1.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls   Avg (ns)   Med (ns)   Min (ns)  Max (ns)   StdDev (ns)          Name        
 --------  ---------------  ---------  ----------  ---------  --------  ---------  -----------  --------------------
     98.3        238638994          3  79546331.3     1110.0       841  238637043  137776597.8  cudaEventCreate     
      1.6          3896582          2   1948291.0  1948291.0    123378    3773204    2580816.7  cudaEventSynchronize
      0.1           163450          3     54483.3    22503.0      3144     137

这时，我们和上一节课的发现程序执行快了很多，并且数据传输的部分 [CUDA memcpy HtoD]  和  [CUDA memcpy DtoH] 不见了

课后作业：
- 尝试利用统一内存和shared memory完成矩阵转置操作

#https://zhuanlan.zhihu.com/p/450242129

#这里是用非统一内存写法，同意内存可以类比修改

先使用老的编译下 ，[matrix_mul_transpose.cu](matrix_mul_transpose.cu)文件，

In [41]:
!make -f Makefile_mlu_transpose
!./matrix_mul_transpose

/usr/local/cuda/bin/nvcc  -arch=compute_80 -code=sm_80 matrix_mul_transpose.cu -o ./matrix_mul_transpose
pass
GPU time is : 0.346400 


In [44]:
!make -f Makefile_mlu_transpose
!./matrix_mul_transpose

make: 'matrix_mul_transpose' is up to date.
pass
GPU time is : 0.378720 
